## 5. 소유권
- 메모리:
	+ 힙 영역: 더 느름, 글로벌하게 접근 가능.
	+ 스택 영역: 더 빠름, 동작에 반드시 필요한 값만 저장하도록 되어있음. 함수가 실행되면 저장, 함수가 끝나면 자동으로 삭제됨 
- 파이썬 메모리 관리:
	+ 모든 데이터를 힙 영역에 저장 -> *가비지 컬렉터*가 자동으로 할당, 해제함. 파이썬이 느린 결정적 이유
- 러스트 메모리 관리:
	+ 스택, 힙 모두 사용
	+ 함수 내부(scope)에서 선언된 값들은 *스택*에 저장하고 사용
	+ global하게 선언된 값들은 *힙*에 저장하고 사용
	+ rust에서 scope는 {}


### 5.1 스택 사용: 스코프
- 러스트의 스코프 영역은 {}
- 함수 내 지역 변수는 {} 기준으로 삭제된다.
- 부모 함수일지라도 자식 함수 스코프에서 사용되면 메모리에서 해제된다.
	+ 자식 함수 스코프 이 외에도 함수가 쓰여야할 경우에 맞춰 소유권 돌려주기를 사용한다.

In [ ]:
fn stack () {
	let var = "string";
	let foo = 1;
	println!("var: {}", var);
	println!("foo: {}", foo);
}

stack();
// println!("var: {}", var);
// println!("foo: {}", foo);
// var, foo는 stack 안에서만 사용 되고 함수 종료 시 메모리 스택에서 삭제됨. 따라서 주석처리된 애들은 에러가 남

var: string


foo: 1


In [ ]:
static var: i32 = 1;

fn main() {
	println!("{}", var+1);
}

main();
println!("{}", var+1);


2
2


### 5.2 소유권 규칙
- 모든 **값**은 해당 값을 **소유**하는 소유자가 있습니다.
- 한번에 하나의 소유자만 있을 수 있습니다. 동시에 한 값에 두 소유자가 있을 수 없습니다.
- 소유자가 현재 코드의 스코프에서 벗어나면 값은 메모리에서 할당해제됩니다.
- rust는 지정한 값의 소유권 유무로 앞으로 더 사용할지, 아니면 사용을 안할지 판단합니다.

In [ ]:
// 스코프 규칙: ok
fn main() {
	let x = 1;
	let y = 2;
	{
		let z = x + y;
		println!("z: {}", z);
	}
	// println!("z: {}", z); # error
}

main();

z: 3


In [5]:
// 스코프 규칙: error
fn main() {
	let x = 1;
	let y = 2;
	{
		let z = x + y;
		// println!("z: {}", z);
	} // 여기서 나오는 순간 z 데이터는 할당해제됨
	println!("z: {}", z);
}

main();

Error: cannot find value `z` in this scope

In [ ]:
// 함수에서 파라미터를 전달할 때도 같은 원리가 적용됨!
fn dummy(x: String) {
	println!("x: {}", x);
}

fn main() {
	let x = String::from("hello");
	dummy(x);
	println!("x: {}", x); // 이 라인 때문에 에러가 남. x는 파라미터 내부에 들어갔고 함수 밖 스코프로 나오는 순간 할당해제됨
}

main();

Error: borrow of moved value: `x`

#### 5.2.1 소유권 돌려주기, 소유권 대여
- 스코프 벗어나도 그 값이 사라지지 않게 하려면?
- 소유권 돌려주기: 메모리에 저장된 값의 소유권을 리턴해 넘겨주는 방식
- 소유권 대여: 소유권을 가져가지 않고 해당 값을 참조하는 방법. **& 키워드 사용**

In [ ]:
// 소유권 돌려주기
// 매번 함수의 값을 리턴해줘야하기 때문에 값이 어느 변수로 이동하는지 알기 어려움

fn dummy(x: String) -> String {
	println!("x: {}", x);
	x // return x 같은 느낌
}

fn main() {
	let x = String::from("hello");
	let x = dummy(x);
	println!("x: {}", x);
}

main();

x: hello
x: hello


In [9]:
fn dummy(y: String) -> String {
	println!("y: {}", y);
	y // return x 같은 느낌
}

fn main() {
	let x = String::from("hello");
	println!("x: {}", x);
	let z = dummy(x);
	println!("z: {}", z);
}

main();

x: hello
y: hello
z: hello


In [10]:
// 레퍼런스와 소유권 대여
// & 키워드 사용
fn dummy(y: &String) {
	println!("y: {}", y);
} // 스코프 밖으로 나오는 순간, 소유권이 x에게 돌아감

fn main() {
	let x = String::from("Hello");
	println!("before fn x: {}", x);
	dummy(&x);
	println!("after fn x: {}", x);
}

main();

before fn x: Hello
y: Hello
after fn x: Hello


In [ ]:
// 소유권 대여한 레퍼런스 값은 기본적으로 변경 불가
// 만약 변경하고 싶으면 가변변수로 먼저 선언하고 그 가변변수를 대여시켜줘야함

fn dummy(y: &mut String) { // 3. 가변변수 대여 -> 함수 인자로 사용
	y.push_str(" World"); // 4. 가변변수 변경
	println!("y: {}", y); // 5. 가변변수 출력
}

fn main() {
	let mut x = String::from("Hello"); //1. 가변변수 선언
	println!("before fn x: {}", x);
	dummy(&mut x); //2. 가변변수 소유권 대여
	println!("after fn x: {}", x); // 5, 변경된 가변변수 확인
}

main();

before fn x: Hello
y: Hello World
after fn x: Hello World


In [16]:
// 가변 레퍼런스로 소유권을 두 곳에 동시에 대여시킬 수 없음

fn main() {
	let mut x = String::from("Hello");
	let y = &mut x;
	let z = &mut x;
	println!("y: {}, z: {}", y, z);
}

main();

Error: cannot borrow `x` as mutable more than once at a time

In [ ]:
// 얜 소유권을 x가 가지고 있고 y, z는 단순히 레퍼런스만 가지고 있으므로 에러 X

fn main() {
	let mut x = String::from("Hello");
	let y = &x; // x의 값만 ref할 뿐, 이 값을 변경시킬 수 없음. 가변 변수를 대여했다고 하더라도!
	let z = &x;
	println!("y: {}, z: {}", y, z);

	let x = String::from("Hello");
	let y = &x;
	let z = &x;
	println!("y: {}, z: {}", y, z);
}

main();

y: Hello, z: Hello
y: Hello, z: Hello


### 5.3 클로저와 소유권
- 클로저는 익명함수
- 클로저는 함수를 변수에 저장하고 함수 자체를 인자로 넘길 수 있음
- 클로저 인자는 클로저 스코프 밖을 나가도 동일한 함수 내부에서 사용 가능

In [21]:
fn main() {
	let multiplier = 5;

	let func = |x: i32| {x * multiplier};

	for i in 1..=5 {
		println!("{}", func(i));
	}
}

main();

5
10
15
20
25


In [ ]:
// 함수에 가변을 주면 환경에 있는 가변변수의 값을 변경시킬 수 있음. 스코프 밖도 변경됨
fn main() {
	let mut multiplier = 5;

	let mut func = |x: i32| {
		multiplier += 1;
		x * multiplier
	};

	for i in 1..=5 {
		println!("{}", func(i));
	}
}

main();

6
14
24
36
50


In [ ]:
// MOVE를 사용한 소유권 이동 가능
// move | param, ... | body;
// 1. factory 변수가 클로저 안에 캡쳐될 때 소유권이 factory -> 클로저로 대여됨
// 2. factory 함수가 끝나면, factor는 메모리 할당 해제됨. factor는 다시 사용할 수 없음
// 3. 방법은 소유권을 클로저 안으로 이동시킴.

fn factory(factor: i32) -> impl Fn(i32) -> i32 {
	| x | x * factor
	// factory는 클로저를 리턴해주는 함수.
	// 클로저의 x는 루프 돌며 들어오는 i.
	// 문제는 factor가 함수에 factory란 함수에 감싸져 있어, 스코프를 벗어나면 할당해제됨. multiplier=5는 해제
	// 반복문은 계속 돈다. 그러니까 x엔 계속 값이 들어오는데, factor(aka. multiplier)는 해제되어 더이상 사용 불가.
	// 해결방법은 소유권을 클로저 안으로 이동시키는 것.
}

fn main() {
	let multiplier = 5;
	let func = factory(multiplier);
	for i in 1..=5 {
		println!("{}", func(i));
	}
}

main();

Error: closure may outlive the current function, but it borrows `factor`, which is owned by the current function

In [ ]:

fn factory(factor: i32) -> impl Fn(i32) -> i32 {
    move | x | x * factor // 
}

fn main() {
    let multiplier = 5;
    let func = factory(multiplier);
    for i in 1..=5 {
        println!("{}", func(i));
    }
}

main();

5
10
15
20
25
